In [1]:
def to_qiskit_hamiltonian(hamiltonian, noq=None):
    """
    
    Input: 
        Pennylane Hamiltonan: hamiltonian
        Number of qubits: noq
    
    Return: 
        QISKit Hamiltonian: qubit_op 
    
    """
    
    from pennylane.operation import Observable, Tensor
    from qiskit.aqua.operators import WeightedPauliOperator
    
    OBS_MAP = {"PauliX": "X", "PauliY": "Y", "PauliZ": "Z", "Hadamard": "H", "Identity": "I"}
    pauli_dict= {"paulis": []}
    
    if len(hamiltonian.coeffs) != len(hamiltonian.ops):
        raise ValueError("Could not create valid Hamiltonian; "
                "number of coefficients and operators does not match.")

    for obs in hamiltonian.ops:
        if not isinstance(obs, Observable):
            raise ValueError("Could not create circuits. Some or all observables are not valid.")             
                
    for i, obs in enumerate(hamiltonian.ops):
        if isinstance(obs, Tensor):
            pauli_term = ["I"] * noq
            for j in obs.obs:
                pauli_term[j.wires[0]] = OBS_MAP[j.name]
        
        elif isinstance(obs, Observable):
            pauli_term = ["I"] * noq
            pauli_term[obs.wires[0]] = OBS_MAP[obs.name]
            
        term = "".join(pauli_term)
        op = {"label": term}
        
        op["coeff"] = {"real": np.real(hamiltonian.coeffs[i]),"imag": np.imag(hamiltonian.coeffs[i])}
             
        pauli_dict["paulis"].append(op)
        
    qubit_op = WeightedPauliOperator.from_dict(pauli_dict)
    
    return qubit_op